Lendo json

In [2]:
import json

path_json = "../../data/raw/modulo_01_pipeline/dados_empresaA.json"

with open(path_json) as file:
    dados_json = json.load(file)

dados_json[0]

{'Nome do Produto': 'Blush em pó',
 'Categoria do Produto': 'Eletrodomésticos',
 'Preço do Produto (R$)': 79.41,
 'Quantidade em Estoque': 7,
 'Filial': 'Filial 7'}

Lendo csv

In [3]:
import csv

path_csv = "../../data/raw/modulo_01_pipeline/dados_empresaB.csv"
dados_csv = []

with open(path_csv) as file:
    spamreader = csv.reader(file, delimiter=",")
    for row in spamreader:
        dados_csv.append(row)

dados_csv[0]

['Nome do Item',
 'Classificação do Produto',
 'Valor em Reais (R$)',
 'Quantidade em Estoque',
 'Nome da Loja',
 'Data da Venda']

Transformando conjunto de listas para uma lista de dicionários

In [4]:
dados_csv = []

with open(path_csv) as file:
    spamreader = csv.DictReader(file, delimiter=",")
    [dados_csv.append(row) for row in spamreader]

dados_csv[0]

{'Nome do Item': 'Lápis de sobrancelha',
 'Classificação do Produto': 'Roupas',
 'Valor em Reais (R$)': '55.17',
 'Quantidade em Estoque': '62',
 'Nome da Loja': 'Filial 1',
 'Data da Venda': '2023-04-13 18:58:06.794203'}

Organizando campos

In [5]:
# Pegar os nomes das colunas do json
nomes_colunas_json = list(dados_json[0].keys())
nomes_colunas_json

['Nome do Produto',
 'Categoria do Produto',
 'Preço do Produto (R$)',
 'Quantidade em Estoque',
 'Filial']

In [6]:
# Pegar os nomes das colunas do csv
nomes_colunas_csv = list(dados_csv[0].keys())
nomes_colunas_csv

['Nome do Item',
 'Classificação do Produto',
 'Valor em Reais (R$)',
 'Quantidade em Estoque',
 'Nome da Loja',
 'Data da Venda']

Renomeando campos

In [7]:
# Mapeando os nomes das colunas do csv para os nomes das colunas do json

key_mapping = {
    "Nome do Item": "Nome do Produto",
    "Classificação do Produto": "Categoria do Produto",
    "Valor em Reais (R$)": "Preço do Produto (R$)",
    "Quantidade em Estoque": "Quantidade em Estoque",
    "Nome da Loja": "Filial",
    "Data da Venda": "Data da Venda",
}
key_mapping

{'Nome do Item': 'Nome do Produto',
 'Classificação do Produto': 'Categoria do Produto',
 'Valor em Reais (R$)': 'Preço do Produto (R$)',
 'Quantidade em Estoque': 'Quantidade em Estoque',
 'Nome da Loja': 'Filial',
 'Data da Venda': 'Data da Venda'}

In [8]:
# Criar uma nova lista de dicionários com os nomes das colunas do json

"""
new_dados_csv = []
for old_dict in dados_csv:
    dict_temp = {}
    for old_key, value in old_dict.items():
        dict_temp[key_mapping[old_key]] = value
    new_dados_csv.append(dict_temp)

new_dados_csv[0]
"""

# Formatado em dictionary comprehension dentro de um list comprehension
new_dados_csv = [
    {key_mapping[old_key]: value for old_key, value in old_dict.items()}
    for old_dict in dados_csv
]
new_dados_csv[0]

{'Nome do Produto': 'Lápis de sobrancelha',
 'Categoria do Produto': 'Roupas',
 'Preço do Produto (R$)': '55.17',
 'Quantidade em Estoque': '62',
 'Filial': 'Filial 1',
 'Data da Venda': '2023-04-13 18:58:06.794203'}

In [9]:
len(dados_json) + len(new_dados_csv)

4446

Juntando os arquivos

In [10]:
combined_list = []
combined_list.extend(dados_json)
combined_list.extend(new_dados_csv)
len(combined_list)

4446

In [11]:
combined_list[-1], combined_list[0]

({'Nome do Produto': 'Sombra de olhos',
  'Categoria do Produto': 'Eletrônicos',
  'Preço do Produto (R$)': '41.73',
  'Quantidade em Estoque': '5',
  'Filial': 'Filial 6',
  'Data da Venda': '2022-11-21 18:58:06.794203'},
 {'Nome do Produto': 'Blush em pó',
  'Categoria do Produto': 'Eletrodomésticos',
  'Preço do Produto (R$)': 79.41,
  'Quantidade em Estoque': 7,
  'Filial': 'Filial 7'})

In [ ]:
# Método get do dicionário para acessar o valor da chave "Data da Venda" do primeiro e do último elemento da lista combinada.
# Retorna nulo se a chave não existir, evitando erros.
combined_list[-1].get("Data da Venda"), combined_list[0].get("Data da Venda")

('2022-11-21 18:58:06.794203', None)

Compartilhar os dados tratados

In [15]:
# Forma estática de adicionar o nome da coluna "Data da Venda" à lista de nomes das colunas do json.
"""
nomes_colunas_json.append("Data da Venda")
nomes_colunas_json
"""

# Forma dinâmica de adicionar o nome da coluna "Data da Venda" à lista de nomes das colunas do json, caso ela não exista.
nomes_colunas = list(combined_list[-1].keys())
nomes_colunas

['Nome do Produto',
 'Categoria do Produto',
 'Preço do Produto (R$)',
 'Quantidade em Estoque',
 'Filial',
 'Data da Venda']

Salvando

In [ ]:
path_dados_combinados = "../../data/processed/modulo_01_pipeline/dados_combinados.csv"

with open(path_dados_combinados, "w") as file:
    writer = csv.DictWriter(file, fieldnames=nomes_colunas_json)
    writer.writeheader()

    [writer.writerow(row) for row in combined_list]

Organizando os dados como uma lista de listas para que fique em um formato mais próximo do csv que será a saída

In [ ]:
dados_combinados_tabela = [nomes_colunas]

for row in combined_list:
    linha = []
    for coluna in nomes_colunas:
        # Retornar indisponível caso a chave não exista, evitando erros
        linha.append(row.get(coluna, "Indisponível"))
    dados_combinados_tabela.append(linha)

In [19]:
dados_combinados_tabela[1], dados_combinados_tabela[-1]

(['Blush em pó', 'Eletrodomésticos', 79.41, 7, 'Filial 7', 'Indisponível'],
 ['Sombra de olhos',
  'Eletrônicos',
  '41.73',
  '5',
  'Filial 6',
  '2022-11-21 18:58:06.794203'])

Salvar dessa nova forma adicionando `Indisponível` nos dados faltantes de `Data de Venda`

In [20]:
path_dados_combinados = "../../data/processed/modulo_01_pipeline/dados_combinados.csv"

with open(path_dados_combinados, "w") as file:
    writer = csv.writer(file)
    writer.writerows(dados_combinados_tabela)